<a href="https://colab.research.google.com/github/TuragDev/ML-Python-Codes/blob/main/Copy_of_CNN_Crystal_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-optimize

In [3]:
pip install keras-tuner

In [4]:
!pip install scikit-learn


In [5]:
pip install keras

In [6]:
pip install tensorflow

In [7]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

In [8]:
pip install keras-tuner

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import joblib
from sklearn.model_selection import train_test_split
import numpy as np

import pandas as pd

from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split

import numpy as np

import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from sklearn.utils import shuffle
from kerastuner.tuners import RandomSearch
from kerastuner.tuners import BayesianOptimization

In [10]:
from kerastuner import HyperParameters

In [11]:
from google.colab import files
uploaded= files.upload()

Saving Orbital_test.csv to Orbital_test (1).csv
Saving Orbital_train.csv to Orbital_train (1).csv
Saving Sine_test.csv to Sine_test (1).csv
Saving Sine_train.csv to Sine_train (1).csv
Saving Xrd_test.csv to Xrd_test (1).csv
Saving Xrd_train.csv to Xrd_train (1).csv


In [ ]:
df= pd.read_csv('Sine_train.csv')
df.shape

(596, 166)

In [ ]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145         cubic
146      trigonal
147     hexagonal
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [ ]:
X_train.shape

(149, 152)

In [ ]:
df=pd.read_csv('Sine_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145         cubic
146      trigonal
147     hexagonal
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [ ]:
X_test.shape

(149, 152)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(152, 1))
    x = inputs

    # Loop for convolutional layers
    for i in range(hp.Int("conv_layers", 1, 3, default=2)):
        x = layers.Conv1D(
            filters=hp.Int("filters_" + str(i), 12, 152, step=12, default=64),
            kernel_size=hp.Choice("kernel_size_" + str(i), [3, 5, 7]),
            activation="relu",
            padding="same",
        )(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Dropout(hp.Float('dropout_conv_' + str(i), 0.3, 0.5, step=0.1, default=0.2))(x)

    x = layers.Flatten()(x)

    # Loop for dense layers
    for i in range(hp.Int("dense_layers", 1, 3, default=1)):
        x = layers.Dense(
            units=hp.Int('dense_units_' + str(i), min_value=12, max_value=152, step=12, default=128),
            activation='relu',
        )(x)
        x = layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.3, 0.5, step=0.1, default=0.3))(x)

    outputs = layers.Dense(len(encoder.categories_[0]), activation="softmax")(x)  # Number of classes

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=150,  # Adjust as needed
    directory='tuner_dir',
    project_name='multiclass_classification'
)

# Start the search
tuner.search(X_train_normalized, y_train_encoded, epochs=20, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(X_test_normalized, y_test_encoded)
print(f"Test accuracy: {test_acc}")

# Summary of the best model
print(best_model.summary())

Trial 17 Complete [00h 00m 04s]
val_accuracy: 0.5

Best val_accuracy So Far: 0.5333333611488342
Total elapsed time: 00h 08m 40s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
3                 |1                 |conv_layers
128               |64                |filters_0
3                 |3                 |kernel_size_0
0.4               |0.4               |dropout_conv_0
128               |32                |filters_1
5                 |3                 |kernel_size_1
0.3               |0.4               |dropout_conv_1
2                 |3                 |dense_layers
64                |128               |dense_units_0
0.5               |0.3               |dropout_dense_0
0.001             |0.01              |learning_rate
32                |128               |dense_units_1
0.4               |0.3               |dropout_dense_1
32                |128               |dense_units_2
0.3               |0.3               |dropout_dense_2
32            

In [ ]:
df= pd.read_csv('Xrd_train.csv')
df.shape

(596, 142)

In [ ]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
591      monoclinic
592        trigonal
593    orthorhombic
594        trigonal
595      monoclinic
Name: crystal_system, Length: 596, dtype: object

In [ ]:
X_train.shape

(596, 128)

In [ ]:
df=pd.read_csv('Xrd_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0          cubic
1          cubic
2      triclinic
3      triclinic
4      triclinic
         ...    
144     trigonal
145        cubic
146     trigonal
147    hexagonal
148        cubic
Name: crystal_system, Length: 149, dtype: object

In [ ]:
X_test.shape

(149, 128)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(128, 1))
    x = inputs

    # Loop for convolutional layers
    for i in range(hp.Int("conv_layers", 1, 7, default=2)):
        x = layers.Conv1D(
            filters=hp.Int("filters_" + str(i), 7, 128, step=12, default=64),
            kernel_size=hp.Choice("kernel_size_" + str(i), [3, 5]),
            activation="relu",
            padding="same",
        )(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Dropout(hp.Float('dropout_conv_' + str(i), 0.3, 0.5, step=0.1, default=0.2))(x)

    x = layers.Flatten()(x)

    # Loop for dense layers
    for i in range(hp.Int("dense_layers", 1, 7, default=1)):
        x = layers.Dense(
            units=hp.Int('dense_units_' + str(i), min_value=7, max_value=128, step=12, default=128),
            activation='relu',
        )(x)
        x = layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.3, 0.5, step=0.1, default=0.3))(x)

    outputs = layers.Dense(len(encoder.categories_[0]), activation="softmax")(x)  # Number of classes

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=500,  # Adjust as needed
    directory='tuner_dir',
    project_name='multiclass_classification'
)

# Start the search
tuner.search(X_train_normalized, y_train_encoded, epochs=20, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(X_test_normalized, y_test_encoded)
print(f"Test accuracy: {test_acc}")

# Summary of the best model
print(best_model.summary())

Trial 390 Complete [00h 00m 16s]
val_accuracy: 0.5416666865348816

Best val_accuracy So Far: 0.6416666507720947
Total elapsed time: 02h 08m 34s

Search: Running Trial #391

Value             |Best Value So Far |Hyperparameter
4                 |3                 |conv_layers
91                |127               |filters_0
5                 |5                 |kernel_size_0
0.4               |0.4               |dropout_conv_0
67                |31                |filters_1
5                 |3                 |kernel_size_1
0.3               |0.5               |dropout_conv_1
1                 |1                 |dense_layers
19                |103               |dense_units_0
0.5               |0.3               |dropout_dense_0
0.01              |0.01              |learning_rate
127               |43                |filters_2
5                 |3                 |kernel_size_2
0.4               |0.5               |dropout_conv_2
19                |31                |filters_3
3       

In [12]:
df= pd.read_csv('Orbital_train.csv')
df.shape

(595, 1038)

In [13]:
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

y_train

0          trigonal
1        tetragonal
2      orthorhombic
3      orthorhombic
4             cubic
           ...     
590    orthorhombic
591        trigonal
592      monoclinic
593       triclinic
594        trigonal
Name: crystal_system, Length: 595, dtype: object

In [14]:
X_train.shape

(595, 1024)

In [15]:
df=pd.read_csv('Orbital_test.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

y_test

0           cubic
1           cubic
2       triclinic
3       triclinic
4       triclinic
          ...    
144    monoclinic
145      trigonal
146     hexagonal
147         cubic
148         cubic
Name: crystal_system, Length: 149, dtype: object

In [16]:
X_test.shape

(149, 1024)

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Data Preprocessing
# Create a scaler object
scaler = StandardScaler()

# Fit scaler on the training data
scaler.fit(X_train)

# Apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert pandas Series to NumPy array and reshape
y_train_array = y_train.to_numpy().reshape(-1, 1)
y_test_array = y_test.to_numpy().reshape(-1, 1)

# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding
y_train_encoded = encoder.fit_transform(y_train_array).toarray()
y_test_encoded = encoder.transform(y_test_array).toarray()

In [18]:
def build_model(hp):
    inputs = keras.Input(shape=(1024, 1))
    x = inputs

    # Loop for convolutional layers
    for i in range(hp.Int("conv_layers", 1, 7, default=2)):
        x = layers.Conv1D(
            filters=hp.Int("filters_" + str(i), 7, 128, step=12, default=64),
            kernel_size=hp.Choice("kernel_size_" + str(i), [3, 5]),
            activation="relu",
            padding="same",
        )(x)
        x = layers.MaxPooling1D(pool_size=2)(x)
        x = layers.Dropout(hp.Float('dropout_conv_' + str(i), 0.3, 0.5, step=0.1, default=0.2))(x)

    x = layers.Flatten()(x)

    # Loop for dense layers
    for i in range(hp.Int("dense_layers", 1, 7, default=1)):
        x = layers.Dense(
            units=hp.Int('dense_units_' + str(i), min_value=7, max_value=128, step=12, default=128),
            activation='relu',
        )(x)
        x = layers.Dropout(hp.Float('dropout_dense_' + str(i), 0.3, 0.5, step=0.1, default=0.3))(x)

    outputs = layers.Dense(len(encoder.categories_[0]), activation="softmax")(x)  # Number of classes

    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [19]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=150,  # Adjust as needed
    directory='tuner_dir',
    project_name='multiclass_classification'
)

# Start the search
tuner.search(X_train_normalized, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(X_test_normalized, y_test_encoded)
print(f"Test accuracy: {test_acc}")

# Summary of the best model
print(best_model.summary())

Trial 150 Complete [00h 01m 29s]
val_accuracy: 0.5210084319114685

Best val_accuracy So Far: 0.5630252361297607
Total elapsed time: 01h 52m 05s
5/5 [==============================] - 1s 44ms/step - loss: 0.9523 - accuracy: 0.6242
Test accuracy: 0.6241610646247864
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1024, 67)          402       
                                                                 
 max_pooling1d (MaxPooling1  (None, 512, 67)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 512, 67)           0         
                                                             